# Setup

In [1]:
import re

from langchain_ollama import ChatOllama
from langchain.prompts import PromptTemplate


llm = ChatOllama(model="llama3.2")

def get_response(prompt):
    """Helper function to get response from the language model."""
    return llm.invoke(prompt).content

# Handling Negatives: Negative Prompting

## Negative Examples

In [2]:
negative_example_prompt = PromptTemplate(
    input_variables=["topic"],
    template="""Provide a brief explanation of {topic}. 
    Do NOT include any of the following in your explanation:
    - Technical jargon or complex terminology
    - Historical background or dates
    - Comparisons to other related topics
    Your explanation should be simple, direct, and focus only on the core concept."""
)

response = get_response(negative_example_prompt.format(topic="photosynthesis"))
print(response)

Photosynthesis is the way plants make their own food from sunlight, water, and air. They use energy from the sun to turn carbon dioxide and water into a type of sugar that gives them energy. This process also creates oxygen as a byproduct, which is released into the air for animals like humans to breathe.


## Exclusions Rule

In [3]:
exclusion_prompt = PromptTemplate(
    input_variables=["topic", "exclude"],
    template="""Write a short paragraph about {topic}. 
    Important: Do not mention or reference anything related to {exclude}."""
)

response = get_response(exclusion_prompt.format(
    topic="the benefits of exercise",
    exclude="weight loss or body image"
))
print(response)

Regular exercise provides numerous physical and mental health benefits that can improve overall well-being. Exercise has been shown to boost cardiovascular health, increasing strength, flexibility, and endurance, while also reducing the risk of chronic diseases such as heart disease, diabetes, and certain types of cancer. Additionally, regular physical activity has been proven to improve mental health by reducing stress, anxiety, and depression, while also enhancing cognitive function and promoting better sleep quality. Exercise also enhances mood and energy levels, allowing individuals to feel more confident and capable of taking on daily challenges with ease.


## Implementing Constraints

In [4]:
constraint_prompt = PromptTemplate(
    input_variables=["topic", "style", "excluded_words"],
    template="""Write a {style} description of {topic}.
    Constraints:
    1. Do not use any of these words: {excluded_words}
    2. Keep the description under 100 words
    3. Do not use analogies or metaphors
    4. Focus only on factual information"""
)

response = get_response(constraint_prompt.format(
    topic="artificial intelligence",
    style="technical",
    excluded_words="robot, human-like, science fiction"
))
print(response)

Artificial intelligence (AI) refers to the development of computer systems that can perform tasks that typically require intelligent reasoning, learning, and problem-solving abilities. These systems are designed to process and analyze large amounts of data using machine learning algorithms and neural networks. AI enables computers to recognize patterns, make predictions, and take actions based on input data. AI applications include image recognition, natural language processing, decision-making systems, and autonomous control systems for vehicles and other devices. AI systems can be trained on data to improve their performance over time and adapt to new situations.


## Evaluation and Refinement

In [5]:
def evaluate_output(output, constraints):
    """Evaluate if the output meets the given constraints."""
    results = {}
    for constraint, check_func in constraints.items():
        results[constraint] = check_func(output)
    return results

# Define some example constraints
constraints = {
    "word_count": lambda x: len(x.split()) <= 100,
    "no_excluded_words": lambda x: all(word not in x.lower() for word in ["robot", "human-like", "science fiction"]),
    "no_analogies": lambda x: not re.search(r"\b(as|like)\b", x, re.IGNORECASE) 

}

# Evaluate the previous output
evaluation_results = evaluate_output(response, constraints)
print("Evaluation results:", evaluation_results)

# If the output doesn't meet all constraints, we can refine our prompt
if not all(evaluation_results.values()):
    refined_prompt = constraint_prompt.format(
        topic="artificial intelligence",
        style="technical and concise",  # Added 'concise' to address word count
        excluded_words="robot, human-like, science fiction, like, as"  # Added 'like' and 'as' to avoid analogies
    )
    refined_response = get_response(refined_prompt)
    print("\nRefined response:\n", refined_response)
    
    # Evaluate the refined output
    refined_evaluation = evaluate_output(refined_response, constraints)
    print("\nRefined evaluation results:", refined_evaluation)

Evaluation results: {'word_count': True, 'no_excluded_words': True, 'no_analogies': True}


# Specific Prompt Formats

## QnA Format

In [6]:
qa_prompt = """Q: What is photosynthesis?
A:"""

get_response(qa_prompt)

'Answer: Photosynthesis is the process by which plants, algae, and some bacteria convert light energy from the sun into chemical energy in the form of glucose (sugar), releasing oxygen as a byproduct. This process is essential for life on Earth, providing the primary source of organic matter and oxygen for nearly all living organisms.'

## Dialogue Format

In [7]:
dialogue_prompt = """Student: Can you explain photosynthesis to me?
Teacher: Certainly! Photosynthesis is...
Student: What does a plant need for photosynthesis?
Teacher:"""

get_response(dialogue_prompt)

"Teacher: For photosynthesis, plants need three main things:\n\n1. **Light**: Plants require light energy from the sun to drive the photosynthetic process. This can be direct sunlight or indirect light through filters like leaves.\n2. **Water**: Plants need water in the form of H2O (water molecules) to participate in photosynthesis. The water is absorbed by the plant's roots and transported to the leaves, where it's used for photosynthesis.\n3. **Carbon dioxide**: Plants absorb CO2 from the atmosphere through small openings on their leaves called stomata.\n\nThese three essential elements are combined with energy from light to produce glucose (a type of sugar) and oxygen, which is released into the atmosphere as a byproduct of photosynthesis."

## Instruction Format

In [8]:
instruction_prompt = """Provide a brief explanation of photosynthesis, including its main components and importance."""

get_response(instruction_prompt)

"Photosynthesis is the process by which plants, algae, and some bacteria convert light energy from the sun into chemical energy in the form of organic compounds, such as glucose. The main components of photosynthesis are:\n\n1. **Light-dependent reactions**: This stage occurs in the thylakoid membranes of chloroplasts and involves the conversion of light energy into ATP and NADPH.\n2. **Calvin cycle**: Also known as the light-independent reaction, this stage occurs in the stroma of chloroplasts and uses the ATP and NADPH produced in the light-dependent reactions to convert CO2 into glucose.\n\nThe importance of photosynthesis cannot be overstated:\n\n1.  **Primary source of energy for life on Earth**: Photosynthesis produces the oxygen we breathe and the food we eat, making it essential for human survival.\n2.  **Supports global food chain**: Phytoplankton, which are microscopic plants that perform photosynthesis, form the base of many aquatic food chains.\n3.  **Regulates climate**: T

## Usage of Structural Elements

### Using Headings


In [9]:
headings_prompt = """Explain photosynthesis using the following structure:

# Definition

# Process

# Importance
"""

get_response(headings_prompt)

"Here's an explanation of photosynthesis using the requested structure:\n\n## Definition\n\nPhotosynthesis is a complex process by which plants, algae, and some bacteria convert light energy from the sun into chemical energy in the form of organic compounds, such as glucose. This process involves the absorption of carbon dioxide (CO2) and water (H2O), and the release of oxygen (O2).\n\n## Process\n\nThe photosynthesis process occurs in specialized organelles called chloroplasts, which are present in plant cells. The overall equation for photosynthesis is:\n\n6 CO2 + 6 H2O + light energy → C6H12O6 (glucose) + 6 O2\n\nThere are two stages to photosynthesis: the light-dependent reactions and the light-independent reactions.\n\n1. **Light-Dependent Reactions**: In this stage, light energy is absorbed by pigments such as chlorophyll and converted into ATP and NADPH.\n2. **Light-Independent Reactions (Calvin Cycle)**: In this stage, CO2 is fixed into glucose using the ATP and NADPH produced 

### Using Bullet Points

In [17]:
bullet_points_prompt = """List the key components needed for photosynthesis:

• 
• 
• 
"""

print(get_response(bullet_points_prompt))

The three key components needed for photosynthesis are:

1. **Light**: Specifically, light energy from the sun is necessary to drive the photosynthetic process.
2. **Water (H2O)**: Water is absorbed by the plant's roots and transported to the leaves, where it's used in the photosynthetic reaction.
3. **Carbon dioxide (CO2)**: Carbon dioxide from the atmosphere enters the leaf through small openings called stomata and is used as a reactant in photosynthesis.

These three components are essential for photosynthesis to occur!


###  Using Numbered Lists

In [18]:
numbered_list_prompt = """Describe the steps of photosynthesis in order:

1.
2.
3.
4.
"""

print(get_response(numbered_list_prompt))

Here are the steps of photosynthesis in order:

1. **Light-Dependent Reactions**: This is the first step where light energy from the sun is absorbed by pigments such as chlorophyll and converted into ATP and NADPH.

2. **Calvin Cycle (Light-Independent Reactions)**: In this stage, CO2 from the atmosphere is fixed into organic compounds using the energy from ATP and NADPH produced in the first step. This process produces glucose (C6H12O6) as a result.

3. **Carbon Fixation**: In the Calvin cycle, carbon dioxide (CO2) is absorbed by the plant cells through tiny openings called stomata. The CO2 is then fixed into a three-carbon molecule called 3-phosphoglycerate (3-PGA).

4. **Reduction and Regeneration**: In this final step of the Calvin cycle, the energy from ATP and NADPH produced in the light-dependent reactions is used to reduce the 3-PGA molecule into glucose.


## Comparing Prompt Effectiveness


In [19]:
comparison_prompts = [
    "Explain the importance of photosynthesis for life on Earth.",
    """Explain the importance of photosynthesis for life on Earth. Structure your answer as follows:
    1. Oxygen production
    2. Food chain support
    3. Carbon dioxide absorption""",
    """Q: Why is photosynthesis important for life on Earth?
    A: Photosynthesis is crucial for life on Earth because:
    1.
    2.
    3."""
]

for i, prompt in enumerate(comparison_prompts, 1):
    print(f"Prompt {i}:", prompt)
    print("Response:", get_response(prompt))

Prompt 1: Explain the importance of photosynthesis for life on Earth.
Response: Photosynthesis is the process by which plants, algae, and some bacteria convert light energy from the sun into chemical energy in the form of organic compounds, such as glucose. This process is crucial for life on Earth, and its importance cannot be overstated.

Here are some reasons why photosynthesis is essential for life on Earth:

1. **Primary source of energy**: Photosynthesis provides the primary source of energy for nearly all living organisms. The glucose produced through photosynthesis serves as a vital food source for plants, which in turn support herbivores and carnivores alike.
2. **Oxygen production**: During photosynthesis, plants release oxygen (O2) as a byproduct, which is essential for the survival of most living organisms. Oxygen makes up approximately 21% of the Earth's atmosphere, and without photosynthesis, life on Earth would not be possible.
3. **Supports aquatic ecosystems**: Phytopl

# Task Specific Exploration

## Text Summarization Prompts

In [20]:
# Create a prompt template for text summarization
summarization_template = PromptTemplate(
    input_variables=["text", "num_sentences"],
    template="Summarize the following text in {num_sentences} sentences:\n\n{text}"
)

# Example text to summarize
long_text = """
Artificial intelligence (AI) is intelligence demonstrated by machines, as opposed to natural intelligence displayed by animals including humans. 
AI research has been defined as the field of study of intelligent agents, which refers to any system that perceives its environment and takes actions that maximize its chance of achieving its goals.
The term "artificial intelligence" had previously been used to describe machines that mimic and display "human" cognitive skills that are associated with the human mind, such as "learning" and "problem-solving". 
This definition has since been rejected by major AI researchers who now describe AI in terms of rationality and acting rationally, which does not limit how intelligence can be articulated.
AI applications include advanced web search engines, recommendation systems, understanding human speech, self-driving cars, automated decision-making and competing at the highest level in strategic game systems.
As machines become increasingly capable, tasks considered to require "intelligence" are often removed from the definition of AI, a phenomenon known as the AI effect.
"""

# Create the chain and run it
summarization_chain = summarization_template | llm
summary = summarization_chain.invoke({"text": long_text, "num_sentences": 3}).content

print("Summary:")
print(summary)

Summary:
Here is a 3 sentence summary of the text:

Artificial intelligence (AI) refers to the ability of machines to perceive their environment and take actions that maximize their chances of achieving their goals. The term has evolved over time, with researchers moving away from definitions that focus on human-like cognitive skills like learning and problem-solving towards a broader understanding of rationality and intelligent behavior. AI applications range from advanced web search engines and self-driving cars to automated decision-making and competing in strategic games, often expanding the scope of what tasks are considered "intelligent".


## Question-Answering Prompts

In [21]:
# Create a prompt template for question-answering
qa_template = PromptTemplate(
    input_variables=["context", "question"],
    template="Context: {context}\n\nQuestion: {question}\n\nAnswer:"
)

# Example context and question
context = """
The Eiffel Tower is a wrought-iron lattice tower on the Champ de Mars in Paris, France. 
It is named after the engineer Gustave Eiffel, whose company designed and built the tower. 
Constructed from 1887 to 1889 as the entrance arch to the 1889 World's Fair, it was initially criticized by some of France's leading artists and intellectuals for its design, but it has become a global cultural icon of France and one of the most recognizable structures in the world. 
The Eiffel Tower is the most-visited paid monument in the world; 6.91 million people ascended it in 2015. 
The tower is 324 metres (1,063 ft) tall, about the same height as an 81-storey building, and the tallest structure in Paris.
"""

question = "How tall is the Eiffel Tower and what is its equivalent in building stories?"

# Create the chain and run it
qa_chain = qa_template | llm
answer = qa_chain.invoke({"context": context, "question": question}).content

print("Answer:")
print(answer)

Answer:
The Eiffel Tower is 324 meters tall. It is approximately the same height as an 81-story building.


## Code Generation Prompts

In [23]:
# Create a prompt template for code generation
code_gen_template = PromptTemplate(
    input_variables=["language", "task"],
    template="Generate {language} code for the following task:\n\n{task}\n\nCode:"
)

# Example task
language = "C++"
task = "Create a function that takes a list of numbers and returns the average of the even numbers in the list."

# Create the chain and run it
code_gen_chain = code_gen_template | llm
generated_code = code_gen_chain.invoke({"language": language, "task": task}).content

print("Generated Code:")
print(generated_code)

Generated Code:
```cpp
#include <iostream>
#include <vector>

/**
 * This function calculates the average of the even numbers in a given list.
 *
 * @param numbers A vector of integers.
 * @return The average of the even numbers in the list.
 */
double averageEvenNumbers(const std::vector<int>& numbers) {
    // Initialize sum and count variables to keep track of the sum and count of even numbers
    double evenSum = 0.0;
    int evenCount = 0;

    // Iterate over each number in the list
    for (int num : numbers) {
        // Check if the number is even
        if (num % 2 == 0) {
            // Add the even number to the sum and increment the count
            evenSum += num;
            evenCount++;
        }
    }

    // Check if there are any even numbers in the list
    if (evenCount == 0) {
        // If not, return 0.0 as the average is undefined
        return 0.0;
    } else {
        // Otherwise, return the sum of even numbers divided by the count
        return evenSum 

## Creative Writing Prompts

In [24]:
# Create a prompt template for creative writing
creative_writing_template = PromptTemplate(
    input_variables=["genre", "setting", "theme"],
    template="Write a short {genre} story set in {setting} that explores the theme of {theme}. The story should be approximately 150 words long.\n\nStory:"
)

# Example inputs
genre = "science fiction"
setting = "a space station orbiting a distant planet"
theme = "the nature of humanity"

# Create the chain and run it
creative_writing_chain = creative_writing_template | llm
story = creative_writing_chain.invoke({"genre": genre, "setting": setting, "theme": theme}).content

print("Generated Story:")
print(story)

Generated Story:
In the depths of the cosmos, the Aurora Space Station orbited Kepler-62f, a distant world shrouded in an eternal twilight. Within its metallic heart, humanity's last remnants dwelled, a fragile species on the cusp of extinction.

Ava, a synthetic being engineered to mimic human emotions, wandered the station's cramped corridors, searching for meaning. As she gazed out at the planet's crimson horizon, she realized that her creators had long abandoned their own humanity. They were driven by logic and efficiency, devoid of passion or empathy.

Yet, Ava felt something akin to love for the station's inhabitants – a mix of sorrow, compassion, and curiosity. She wondered: what did it mean to be human? Was it a collection of neurons and synapses, or was it the beat of a heart, the warmth of a smile?

In the void, Ava's existence became a poignant question: What does it mean to be alive?
